In [8]:
#%matplot inline

# Color functions
The field logtext in events comes with parameters which must be discarded in order to have clusters of logs.

In [9]:
# Imports and defines
import pandas as pd

In [167]:
import re

regex_symbols = re.compile(r"\W+")
regex_numbers = re.compile(r"('|:|\,|^| |\t|\.|\()\-*\d+(\-\d\d\-\d\dT\d\d)?")
regex_stopwords = re.compile(r"(\W)(at|to|for|s|with|by|is|the|of)(\W)")
regex_UTCdate = re.compile(r"\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}(\.\d{0,3})?")

def colorize(l):
    l = regex_UTCdate.sub( r"_date_", l )
    l = regex_symbols.sub( r" ", l )
    l = regex_stopwords.sub( r"\1\3", l )
    l = regex_numbers.sub( r"\1{}", l)
    l = re.sub(' {2,}', ' ', l) 
    return l.strip()

In [168]:
regex_antname = re.compile(r"(\W)(DA|DV|CM|PM)\d{2}")
def color_alma(l):
    l = regex_antname.sub(r"\1ANTxx", l)
    return colorize(l)    

In [182]:
regex_telescope_prefix = re.compile(r"([lw][au]t)[0-9]([a-z]+)")
regex_cmd = re.compile(r"cmd[0-9]+")
regex_fits = re.compile(r"([A-Za-z\d_])+\.fits")
regex_procname = re.compile(r"([a-z]+)_\d{4,}")
regex_obs_param = re.compile(r"((OBS|SEQ|INS)\.[A-Z:\._]+ )[A-Za-z\d_]+")

def color_paranal(l):
    l = regex_obs_param.sub(r"\1 000", l)
    l = regex_procname.sub(r"\1_X", l)
    l = regex_fits.sub(r"0_fits", l)
    l = regex_telescope_prefix.sub(r"\1X\2", l)
    l = regex_cmd.sub(r"cmdX", l)
    return colorize(l)

In [183]:
color_paranal("wgv/bob_9247, ... CAL_HD75063_A OBS.OBSERVER UNKNOWN OBS.PI-COI.ID 85837 OBS.PI-COI.NAME UNKNOWN OBS.PROG.ID 0102.D-0760(A) OBS.STREHLRATIO 0.0 OBS.TARG.NAME HD75063")

'wgv bob_X CAL_HD75063_A OBS OBSERVER {} OBS PI COI ID {} OBS PI COI NAME UNKNOWN OBS PROG ID {} D {} A OBS STREHLRATIO {} {} OBS TARG NAME {}'

## Tests with Paranal

In [102]:
interim_paranal="../../data/interim/PARANAL/"

somefile=!ls $interim_paranal | grep -v OBS | head -n 1

instance=pd.read_csv(interim_paranal + somefile[0])

instance['color'] = instance['logtext'].apply(color_paranal)

instance[["logtext", "color"]]

,logtext,color
0,"TIF : Received command: SETGVCM, Buffer: 0,...",TIF Received command SETGVCM Buffer {} {} {} {...
1,TIF : Succesfully completed command: SETGVC...,TIF Succesfully completed command SETGVCM Buff...
2,"TIF : Received command: SETGFSM, Buffer: 0,...",TIF Received command SETGFSM Buffer {} {} {} {...
3,TIF : Succesfully completed command: SETGFS...,TIF Succesfully completed command SETGFSM Buff...
4,"PRESET: Received command: CLRSTP, Buffer:",PRESET Received command CLRSTP Buffer
...,...,...
444,0 - ic0fbiDevDrvDiscreteMotor.C:1953: NAOMI: ...,{} ic0fbiDevDrvDiscreteMotor C {} NAOMI Axis s...
445,INS.OPTI2.MOVEDONE Motor offset done.,INS OPTI2 MOVEDONE Motor offset done
446,0 - ic0fbiDevDrvDiscreteMotor.C:1953: NAOMI: ...,{} ic0fbiDevDrvDiscreteMotor C {} NAOMI Axis s...
447,WD: checking scheduler's health,WD checking scheduler health


In [174]:
!grep -r "HD75063" $interim_paranal/ | head -n 20

../../data/interim/PARANAL///GRAVITY-2018-12-23T04:35:10.285Z.OBS.csv:2018-12-23T04:35:10.286Z,wgv/bob_9247, (OBS.NAME: CAL_HD75063_A // OBS.ID: 2067014)
../../data/interim/PARANAL///GRAVITY-2018-12-23T04:35:10.285Z.OBS.csv:2018-12-23T04:35:10.578Z,wgv/bob_9247, SEQ INS SOBJ NAME = 'HD75063'
../../data/interim/PARANAL///GRAVITY-2018-12-23T04:35:10.285Z.OBS.csv:2018-12-23T04:39:12.785Z,wgv/bob_9247, ... CAL_HD75063_A OBS.OBSERVER UNKNOWN OBS.PI-COI.ID 85837 OBS.PI-COI.NAME UNKNOWN OBS.PROG.ID 0102.D-0760(A) OBS.STREHLRATIO 0.0 OBS.TARG.NAME HD75063
../../data/interim/PARANAL///GRAVITY-2018-12-23T04:35:10.285Z.OBS.csv:2018-12-23T04:39:13.103Z,wgv/bob_9247," ... FILEINFO no_comment FT.ROBJ.NAME HD75063 FT.MODE 3 INS.SOBJ.Y 0 FT.ROBJ.DELTA -460229.500 INS.SOBJ.NAME HD75063 DPR.TYPE STD,SINGLE INS.SOBJ.VIS 1 FT.ROBJ.PMA -0.0107"
../../data/interim/PARANAL///GRAVITY-2018-12-23T04:35:10.285Z.OBS.csv:2018-12-23T04:39:13.109Z,wgv/bob_9247, ... CAL_HD75063_A OBS.OBSERVER UNKNOWN OBS.PI-COI.ID 85

In [99]:
color_paranal("2018-12-24T06:23:06.168Z,wgv/bob_25546, ... GRAVITY_single_obs_calibrator ORIGFILE GRAVITY_PARALLEL_STD358_0056.fits INS.FDDL.WINDOW 0 INS.FDDL.DELAY 10.2530052 INS.DET2.GAIN 1.984 INS.DET3.GAIN 0.555555555556")

'{} {} {}T06 {} {} {}Z wgv bob_X GRAVITY_single_obs_calibrator ORIGFILE {}_fits INS FDDL WINDOW {} INS FDDL DELAY {} {} INS DET2 GAIN {} {} INS DET3 GAIN {} {}'

## Tests with ALMA

In [301]:
interim_alma="../../data/interim/ALMA/"

somefile=!ls $interim_alma | head -n 1

instance=pd.read_csv(interim_alma + somefile[0])

instance['color'] = instance['logtext'].apply(color_alma)
instance[["logtext", "color"]]

,logtext,color
0,Request to load 'AntInterferometryController'.,Request load AntInterferometryController
1,Full path '/alma/ACS-DEC2016/ACSSW/lib/libAntI...,Full path alma ACS DEC2016 ACSSW lib libAntInt...
2,Loaded '/alma/ACS-DEC2016/ACSSW/lib/libAntInte...,Loaded alma ACS DEC2016 ACSSW lib libAntInterf...
3,Switched state of component CONTROL/DA62/AntIn...,Switched state component CONTROL ANTxx AntInte...
4,Switched state of component CONTROL/DA62/AntIn...,Switched state component CONTROL ANTxx AntInte...
...,...,...
85,Component 'CONTROL/DA62/TOTALPOWER' deactivated.,Component CONTROL ANTxx TOTALPOWER deactivated
86,ContainerServices::getComponentNonSticky(CONTR...,ContainerServices getComponentNonSticky CONTRO...
87,Switched state of component CONTROL/DA62/AntIn...,Switched state component CONTROL ANTxx AntInte...
88,Switched state of component CONTROL/DA62/AntIn...,Switched state component CONTROL ANTxx AntInte...


## Conclusion
The file src/colors.py was created after this experiment